In [1]:
#importing pertinent libraries
import re
import os
import csv
import sys
sys.path.append('../src/')
sys.path.append('../db/')
sys.path.append('../queries/')
import bs4
from bs4 import BeautifulSoup, NavigableString, Tag
import time
import random
import pandas as pd
import requests
from itertools import batched
import ufcInsertQueries as iq
import fightDetailsExtractor as f
import eventDetailsExtractor as e
import pgconnect as pg
import configparser

%load_ext autoreload
%autoreload 2

In [2]:
# Load config
config = configparser.ConfigParser()

config.read("../config.ini")

['../config.ini']

In [3]:
with open('../data/ingested_event_urls.csv', 'r') as file:
    ingested_events_list = file.readlines()
    ingested_events_list = [item.strip() for item in ingested_events_list]

ingested_events_list = ingested_events_list[0].split(',')
ingested_events_list = list(reversed(ingested_events_list))

In [7]:
for index, event in enumerate(ingested_events_list[:2]):

    #extracting fights listed on event
    event_id = index + 1

    #identifying fights on current event card
    fight_url_list = e.getEventFightUrls(event)
    
    for fight_url in fight_url_list[:2]:

        #retrieving fight data
        fight_response = requests.get(fight_url)

        #converting response to bs4 object
        fight_soup = BeautifulSoup(fight_response.content, 'html.parser')

        #extracting fight data
        fight_data_dict = f.extractFightData(fight_soup)

        #identifying data for fights table
        fighter_rc_name, fighter_bc_name = fight_data_dict.get('matchup')
    
        fight_winner = fight_data_dict.get('winner')
        
        fight_weight_class = fight_data_dict.get('weight_class')
    
        method, ending_round, ending_time, time_format, referee, details = fight_data_dict.get('outcome')

        #extracting lists for summary and round by round data
        
        fight_total_summ_stats, fight_rbr_stats, fight_sig_strks_summ_stats, fight_rbr_sig_strks_stats = fight_data_dict.get('fight_stats_list')

        #creating final data list for significant strikes summary stats
        fight_sig_strks_summ_stats.extend(fight_data_dict.get('sig_strks_list'))

        fight_total_summ_stats = [item.replace('%', '') if item != None else item for item in fight_total_summ_stats]
        fight_rbr_stats = [item.replace('%', '') if item != None else item for item in fight_rbr_stats]
        fight_sig_strks_summ_stats = [item.replace('%', '') if item != None else item for item in fight_sig_strks_summ_stats]
        fight_rbr_sig_strks_stats = [item.replace('%', '') if item != None else item for item in fight_rbr_sig_strks_stats] 

        ufc_conn = pg.ufcConnect(config)
        cur = ufc_conn.cursor()
        
        try:
            cur.execute(
                        iq.fights_table_insert_query,
                        (                                                                                  
                            event_id,
                            fighter_rc_name,
                            fighter_bc_name,
                            fight_winner,
                            fight_weight_class,
                            method, 
                            ending_round,
                            ending_time,
                            time_format,
                            referee,
                            details
                        )
                       )
            cur.execute("SELECT max(id) FROM fights")
            
            fight_id = cur.fetchone()[0]

            fight_total_summ_stats.insert(0, fight_id)
            
            cur.execute(iq.fight_total_summ_stats_raw_table_insert_query, fight_total_summ_stats)
            
            fight_sig_strks_summ_stats.insert(0, fight_id)

            cur.execute(iq.fight_sig_strks_summ_stats_raw_table_insert_query, fight_sig_strks_summ_stats)
            
            chunk_size = 18
        
            rbr_data_lists = list(batched(fight_rbr_stats, chunk_size))
            
            for idx, data_tuple in enumerate(rbr_data_lists):
                rnd_num = idx + 1
                data_list = list(data_tuple)
                data_list.insert(0, rnd_num)
                data_list.insert(0, fight_id)
                
                cur.execute(iq.fight_rbr_stats_raw_table_insert_query, data_list) 

            # fight_rbr_sig_strks_stats_raw
            rbr_sig_strks_data_lists = list(batched(fight_rbr_sig_strks_stats, 16))

            for idx, data_tuple in enumerate(rbr_sig_strks_data_lists):
                rnd_num = idx + 1
                data_list = list(data_tuple)
                data_list.insert(0, rnd_num)
                data_list.insert(0, fight_id)

                cur.execute(iq.fight_rbr_sig_strks_stats_raw_table_insert_query, data_list) 

        finally:
            
            ufc_conn.commit()
            
            cur.close()

        sleep_duration = random.uniform(1, 5)

        time.sleep(sleep_duration)
        